In [ ]:
import numpy as np
import scipy.io as scio
import sys
sys.path.insert(0,'../../python/caffe')
import caffe
caffe.set_mode_gpu()
import lmdb
train_data=scio.loadmat('train.mat')
train_label=scio.loadmat('train_label.mat')
val_data=scio.loadmat('val.mat')
val_label=scio.loadmat('val_label.mat')
train_data=train_data['traindata']
train_label=train_label['train_label']
val_data=val_data['valdata']
val_label=val_label['val_label']
stack_train_label=np.hstack((train_data,train_label))
stack_val_label=np.hstack((val_data,val_label))
np.random.shuffle(stack_train_label)
np.random.shuffle(stack_val_label)
train_data=stack_train_label[:,:-1]
train_label=stack_train_label[:,-1]
val_data=stack_val_label[:,:-1]
val_label=stack_val_label[:,-1]
def dataToLmdb(lmdbname,traindata,labels=None):
    env=lmdb.open(lmdbname,map_size=traindata.nbytes*2)
    for i in range(len(traindata)):
        datum=caffe.proto.caffe_pb2.Datum()
        datum.channels=1
        datum.height=1
        datum.width=1
        if traindata.dtype == np.int:
            datum.data = traindata[i].tostring()
        elif traindata.dtype == np.float: 
            datum.float_data.extend(traindata[i].flat)
        else:
            raise Exception("features.dtype unknown.")
        if labels is not None:
            datum.label=int(labeldata[i])
        str_id = '{:08}'.format(i)
        with env.begin(write=True) as txn:
            txn.put(str_id, datum.SerializeToString())
dataToLmdb('train-lmdb',train_data,train_label)
dataToLmdb('val-lmdb',val_data,val_label)